<a href="https://colab.research.google.com/github/Ignvz/copert_5/blob/main/Factores_emision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd "/content/drive/MyDrive/Factores de emision"
!ls

/content/drive/.shortcut-targets-by-id/1rXzOeDHO59D46y_UeQ5BIKirkwBHax7H/Factores_de_emision
'1.A.3.b.i-iv Road Transport Appendix 4 Emission Factors_2020Sept.xlsx'
'Actividad Vehicular.csv'
 BC_emissions.xlsx
 BC_factors.csv
 C5_reducido_prueba.xlsx
 C5_reducido.xlsx
 CH4_emissions.xlsx
 CH4_factors.csv
 CH4_reducido.xlsx
 CO2_emissions.xlsx
 CO2_factors.csv
 CO2_factors.gsheet
 CO2_reducido.xlsx
 CO_emissions.xlsx
 CO_factors.csv
 Copert5.xlsx
 CO_reducido.xlsx
 Euro_7.xlsx
 NOx_emissions.xlsx
 NOx_factors.csv
 NOx_reducido.xlsx
 PM_emissions.xlsx
 PM_factors.csv
 PM_reducido.xlsx
 SOx_emissions.xlsx
 SOx_factors.csv
 Velocidades.csv
 VOC_emissions.xlsx
 VOC_factors.csv
 VOC_reducido.xlsx


In [5]:
import pandas as pd
import numpy as np

In [15]:
def copert5(alpha, beta, gamma, delta, epsilon, hta, vm, redFactor):
  if epsilon*vm**2+2*vm+hta == 0:
    return 0
  else:
    return (1-redFactor)*(alpha*vm**2 + beta*vm + gamma + delta/vm)/(epsilon * vm**2 + 2*vm+ hta)

In [6]:
#cargando. excels
xls = pd.ExcelFile('Copert5.xlsx')
df = pd.read_excel(xls,'HOT_EMISSIONS_PARAMETERS', index_col=None)

In [ ]:
#Reduciendo COPERT 5 Para utilizar solo las secciones necesarias
df = df.loc[
       (df['Segment'] == 'Medium' ) | #VLP, TAX
       (df['Segment'] == 'N1-II' ) | #VLC
       (df['Segment'] == ('Rigid <=7,5 t' , 'Rigid 12 - 14 t')) | #CAM LIV
       (df['Segment'] == 'Rigid 12 - 14 t' ) | #CAM MED
       (df['Segment'] == 'Rigid 20 - 26 t' ) | #CAM PES
       (df['Segment'] == 'Urban Buses Standard 15 - 18 t' ) | #BUS RIG
       (df['Segment'] == 'Urban Buses Standard >18 t' ) | #BUS ART
       (df['Segment'] == 'Coaches Standard <=18 t' ) | #BUS INT PROV
       (df['Segment'] == 'Motorcycles 2-stroke >50 cm³' ) #MOT
       ]
df.to_excel('C5_reducido.xlsx',index = False)


In [19]:
#Filtrando usando REGEX a las categorías a utilizar
df = df.loc[
      df.Segment.str.contains('^Medium') | #VLP, TAX
      df.Segment.str.contains("^N1-II")  | #VLC
      df.Segment.str.contains('^Art*|^Rigid 28 *|^Rigid >32*') | #CAM PES RIGID>28 t and articulated
      df.Segment.str.contains('^Rigid 14 *|^Rigid 20 *|^Rigid 26 *') |#CAM MED RIGID 14-28 t
      df.Segment.str.contains('^Rigid <=7,5 *|^Rigid 7,5 -*|^Rigid 12 *') |#CAM LIV
      df.Segment.str.contains('^Urban Buses Standard 15 *') | #Bus RIG
      df.Segment.str.contains('^Urban Buses Standard >18 t') | #BUS ART
      df.Segment.str.contains('^Coaches Standard <=18 t') | #BUS INT PROV
      df.Segment.str.contains("^Motorcycles 2-stroke >50 *") #MOT
]

In [17]:
#Cambiando a la clasificacion a utilizar
df.loc[df['Segment'] == 'Medium', 'Segment'] = 'VLP'
#df.loc[df['Segment'] == 'N1-II', 'Segment'] = 'VLC'
#df.loc[df['Segment'] == 'Rigid <=7,5 t', 'Segment'] = 'CAM LIV'
#df.loc[df['Segment'] == 'Rigid 12 - 14 t', 'Segment'] = 'CAM MED'
#df.loc[df['Segment'] == 'Rigid 20 - 26 t', 'Segment'] = 'CAM PES'
#df.loc[df['Segment'] == 'Urban Buses Standard 15 - 18 t', 'Segment'] = 'BUS RIG'
#df.loc[df['Segment'] == 'Urban Buses Standard >18 t', 'Segment'] = 'BUS ART'
#df.loc[df['Segment'] == 'Coaches Standard <=18 t', 'Segment'] = 'BUS INT PROV'
#df.loc[df['Segment'] == 'Motorcycles 2-stroke >50 cm³', 'Segment'] = 'MOT'
#df.to_excel('C5_reducido.xlsx',index = False)


,Category,Fuel,Segment,Euro Standard,Technology,Pollutant,Mode,,Load,Min Speed [km/h],...,Beta,Gamma,Delta,Epsilon,Zita,Hta,Reduction Factor [%],Bio Reduction Factor [%],15.0,EF [g/km] or ECF [MJ/km]
0,Passenger Cars,Petrol,Mini,Euro 4,GDI,CO,NaN,NaN,NaN,5.0,...,-0.033418,5.109835,-1.043727e-07,1.871536e-03,-5.288309e-01,37.505739,0.0,0.0,15,0.153648
1,Passenger Cars,Petrol,Mini,Euro 4,GDI,NOx,NaN,NaN,NaN,5.0,...,-0.008580,0.577346,1.306531e-12,2.701764e-17,-1.307669e-13,5.430520,0.0,0.0,15,0.084213
2,Passenger Cars,Petrol,Mini,Euro 4,GDI,VOC,NaN,NaN,NaN,5.0,...,-0.000139,0.047375,-9.097745e-14,-6.441796e-15,7.726315e-13,4.014853,0.0,0.0,15,0.011478
3,Passenger Cars,Petrol,Mini,Euro 4,GDI,PM Exhaust,Urban Peak,NaN,NaN,10.0,...,0.000000,0.001280,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,0.0,0.0,15,0.001280
4,Passenger Cars,Petrol,Mini,Euro 4,GDI,PM Exhaust,Urban Off Peak,NaN,NaN,10.0,...,0.000000,0.001280,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,0.0,0.0,15,0.001280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28985,L-Category,Diesel,Micro-car,Euro 5,NaN,EC,NaN,NaN,NaN,10.0,...,0.000000,1.135618,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,0.0,0.0,15,1.135618
28986,L-Category,Diesel,Micro-car,Euro 5,NaN,CH4,Urban Peak,NaN,NaN,10.0,...,0.000000,0.075000,0.000000e+00,0.000000e+00,0.000000e+00,1000.000000,0.0,0.0,15,0.000075
28987,L-Category,Diesel,Micro-car,Euro 5,NaN,CH4,Urban Off Peak,NaN,NaN,10.0,...,0.000000,0.075000,0.000000e+00,0.000000e+00,0.000000e+00,1000.000000,0.0,0.0,15,0.000075
28988,L-Category,Diesel,Micro-car,Euro 5,NaN,CH4,Rural,NaN,NaN,10.0,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1000.000000,0.0,0.0,15,0.000000


In [ ]:
df = pd.read_excel('C5_reducido.xlsx')
df = df.loc[
              #Combustible
              ((df['Fuel'] == 'Petrol') |  (df['Fuel'] == 'Diesel')) &
              #Euro
              (
              (df['Euro Standard'] == 'PRE ECE') |  
              (df['Euro Standard'] == 'Euro 1') |
              (df['Euro Standard'] == 'Euro 2') |
              (df['Euro Standard'] == 'Euro 3') |
              (df['Euro Standard'] == 'Euro 4') |
              (df['Euro Standard'] == 'Euro 5') |
              (df['Euro Standard'] == 'Euro 6 d') | 
              (df['Euro Standard'] == 'Conventional') |
              (df['Euro Standard'] == 'Euro I') |
              (df['Euro Standard'] == 'Euro II') |
              (df['Euro Standard'] == 'Euro III') |
              (df['Euro Standard'] == 'Euro IV') |
              (df['Euro Standard'] == 'Euro V') |
              (df['Euro Standard'] == 'Euro VI') |
              (df['Euro Standard'] == 'Euro VI D/E')
              )
              ]
df.to_excel('C5_reducido_prueba.xlsx',index = False)

In [ ]:
#Cambiando a la clasificacion a usar
df.loc[df['Euro Standard'] == 'PRE ECE', 'Euro Standard'] = 'No cat'
df.loc[df['Euro Standard'] == 'Euro 1', 'Euro Standard'] = 'E1'
df.loc[df['Euro Standard'] == 'Euro 2', 'Euro Standard'] = 'E2'
df.loc[df['Euro Standard'] == 'Euro 3', 'Euro Standard'] = 'E3'
df.loc[df['Euro Standard'] == 'Euro 4', 'Euro Standard'] = 'E4'
df.loc[df['Euro Standard'] == 'Euro 5', 'Euro Standard'] = 'E5'
df.loc[df['Euro Standard'] == 'Euro 6 d', 'Euro Standard'] = 'E6'
df.loc[df['Euro Standard'] == 'Conventional', 'Euro Standard'] = 'No cat'
df.loc[df['Euro Standard'] == 'Euro I', 'Euro Standard'] = 'E1'
df.loc[df['Euro Standard'] == 'Euro II', 'Euro Standard'] = 'E2'
df.loc[df['Euro Standard'] == 'Euro III', 'Euro Standard'] = 'E3'
df.loc[df['Euro Standard'] == 'Euro IV', 'Euro Standard'] = 'E4'
df.loc[df['Euro Standard'] == 'Euro V', 'Euro Standard'] = 'E5'
df.loc[df['Euro Standard'] == 'Euro VI', 'Euro Standard'] = 'E6'
df.loc[df['Euro Standard'] == 'Euro VI D/E', 'Euro Standard'] = 'E6'
df.loc[df['Fuel'] == 'Petrol', 'Fuel'] = 'Gasolina'
df.to_excel('C5_reducido_prueba.xlsx',index = False)


In [ ]:
df = pd.read_excel('C5_reducido_prueba.xlsx')
df = df.loc[
            #Todo lo en blanco
            (df['Technology'].isnull()) |
            #Passenger Cars
            ((df['Category'] == 'Passenger Cars') & 
             (df['Technology'] == 'GDI') | (df['Technology'] == 'DPF')) |
            #Light Commercial Vehicles
            ((df['Category'] == 'Light Commercial Vehicles') & 
             (df['Technology'] == 'PFI') | (df['Technology'] == 'DPF')) |
             #Buses
            ((df['Category'] == 'Buses') & 
             (df['Technology'] == 'SCR') | (df['Technology'] == 'DPF+SCR')) |
            #Heavy Duty Trucks
            ((df['Category'] == 'Heavy Duty Trucks') & 
             (df['Technology'] == 'SCR') | (df['Technology'] == 'DPF+SCR'))
            ]
            #Motorcycles no requieren filtro
df.to_excel('C5_reducido_prueba.xlsx',index = False)
#Se borra la columna Road Slope idk

In [ ]:
df = pd.read_excel('C5_reducido_prueba.xlsx')
df = df.loc[ ((df['Road Slope'].isnull()) & (df['Load'].isnull())) |
       ((df['Road Slope'] == 0) & (df['Load'] == 0.5))        
            ]
df.to_excel('C5_reducido_prueba.xlsx',index = False)

In [ ]:
#Una vez reducida a lo que se utilizara, se separan dataframes diferentes por contaminante.
df = pd.read_excel('C5_reducido_prueba.xlsx')
cols = list(df.columns)
df = df[cols[0:9]+cols[11:19]]
dfCO = df.loc[df['Pollutant'] == 'CO']
dfNOx = df.loc[df['Pollutant'] == 'NOx']
dfVOC = df.loc[df['Pollutant'] == 'VOC']
dfPM = df.loc[df['Pollutant'] == 'PM Exhaust']
dfCH4 = df.loc[df['Pollutant'] == 'CH4']
dfCO2 = df.loc[df['Pollutant'] == 'EC']
dfCO.to_excel('CO_reducido.xlsx',index = False)
dfNOx.to_excel('NOx_reducido.xlsx',index = False)
dfVOC.to_excel('VOC_reducido.xlsx',index = False)
dfPM.to_excel('PM_reducido.xlsx',index = False)
dfCH4.to_excel('CH4_reducido.xlsx',index = False)
dfCO2.to_excel('CO2_reducido.xlsx',index = False)

In [ ]:
#Toques finales a CH4, PM exhaust
Archivos = ['CH4_reducido.xlsx','PM_reducido.xlsx']
for file in Archivos:
  df = pd.read_excel(file)
  df = df.loc[
              df['Mode'].isnull() |
              (df['Mode'] == 'Highway') |
              (df['Mode'] == 'Urban Off Peak')
              ]
  df.loc[df['Mode'] == 'Highway', 'Mode'] = 'Interurbano'
  df.loc[df['Mode'] == 'Urban Off Peak', 'Mode'] = 'Urbano'
  df.to_excel(file,index = False)